In [ ]:
DATASET_NAME = 'Flowers-101'
datasets_datastore_uri = "azureml://subscriptions/dbd697c3-ef40-488f-83e6-5ad4dfb78f9b/resourcegroups/rbhimani-rg/workspaces/rbhimani-eastus/datastores/datasets"
jsonl_path = f'Benchmarks/{DATASET_NAME}/Benchmark_TEST_MC.jsonl'

In [ ]:
from azureml.fsspec import AzureMachineLearningFileSystem
import pandas as pd

fs = AzureMachineLearningFileSystem(datasets_datastore_uri)
df = pd.read_json(datasets_datastore_uri + "/paths/" + jsonl_path, lines=True)

In [ ]:
# get the text input
text_labels_array = df['label'].unique()
text_labels = ','.join(text_labels_array)
print(text_labels)

In [ ]:
# decode images
import base64
import os
def read_image(fs, image_path):
    with fs.open(image_path, "rb") as f:
        return f.read()

data_out = []
for index, row in df.iterrows():
    sample = {}
    image_dir = './data'
    datastore_image_path = row['image_url'].split(DATASET_NAME)[1]
    image_path = datasets_datastore_uri + '/paths/Benchmarks/' + DATASET_NAME + datastore_image_path
    sample['image'] = base64.b64encode(read_image(fs, image_path)).decode("utf-8")
    sample['text'] = text_labels
    sample['label'] = row['label']
    data_out.append(sample)

In [ ]:
# write processed jsonl file
import json
OUTPUT_FILE_NAME = os.path.join("./output_jsonls", f'{DATASET_NAME}_processed.jsonl')
with open(OUTPUT_FILE_NAME, 'w') as f:
    for sample in data_out:
        f.write(json.dumps(sample) + '\n')